In [1]:
import numpy as np
import cv2 as cv
import os
import json
from PIL import Image
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import time
import torch
from pathlib import Path

from lac.perception.segmentation import Segmentation
from lac.perception.depth import stereo_depth_from_segmentation, project_pixel_to_rover, compute_rock_coords_rover_frame, compute_rock_radii
from lac.control.controller import ArcPlanner
from lac.utils.visualization import overlay_mask, overlay_stereo_rock_depths
from lac.utils.plotting import (
    plot_points_rover_frame,
    plot_path_rover_frame,
    plot_rocks_rover_frame,
)
from lac.utils.frames import invert_transform_mat
from lac.util import load_data, get_positions_from_poses
import lac.params as params

%load_ext autoreload
%autoreload 2

/home/navlab-aa290/miniconda3/envs/lac/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/navlab-aa290/miniconda3/envs/lac/lib/python3.10/site-packages/sam2/modeling/sam/transformer.py:23: UserWarning: Flash Attention is disabled as it requires a GPU with Ampere (8.0) CUDA capability.
  OLD_GPU, USE_FLASH_ATTN, MATH_KERNEL_ON = get_sdpa_settings()


## Obstacle detection


In [3]:
segmentation = Segmentation()

/home/navlab-aa290/miniconda3/envs/lac/lib/python3.10/site-packages/torch/utils/cpp_extension.py:1965: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
Could not load the custom kernel for multi-scale deformable attention: Error building extension 'MultiScaleDeformableAttention': [1/2] /usr/bin/nvcc --generate-dependencies-with-compile --dependency-output ms_deform_attn_cuda.cuda.o.d -DTORCH_EXTENSION_NAME=MultiScaleDeformableAttention -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -I/home/navlab-aa290/miniconda3/envs/lac/lib/python3.10/site-packages/transformers/kernels/deformable_detr -isystem /home/navlab-aa290/miniconda3/envs/lac/lib/python3.10/site-packages/torch/include -isystem /home/navlab-aa290/miniconda3/envs/lac/lib/python3.10/site-packages/torch

In [4]:
data_path = Path("../../output/AutoAgent/arc_planner_test")
initial_pose, lander_pose, poses, imu_data, cam_config = load_data(data_path)

In [5]:
i = 1080

current_pose = poses[i]

In [ ]:
left_image = Image.open(data_path / "FrontLeft" / f"{i}.png")
right_image = Image.open(data_path / "FrontRight" / f"{i}.png")

left_seg_masks, left_seg_full_mask = segmentation.segment_rocks(left_image.convert("RGB"))
right_seg_masks, right_seg_full_mask = segmentation.segment_rocks(right_image.convert("RGB"))

results = stereo_depth_from_segmentation(
    left_seg_masks, right_seg_masks, params.STEREO_BASELINE, params.FL_X
)
left_overlay = overlay_mask(np.array(left_image), left_seg_full_mask)
left_overlay = overlay_stereo_rock_depths(left_overlay, results)
fig = plt.figure(figsize=(10, 5), dpi=100)
plt.imshow(left_overlay)
plt.axis("off")

In [2]:
# TODO: use compute_rock_points and compute_rock_radii

In [ ]:
x, _, w, _ = cv.boundingRect(results[0]["left_mask"].astype(np.uint8))
width_x = w * results[0]["depth"] / params.FL_X
print(f"Width: {width_x} m")

In [ ]:
rock_points_rover_frame = []
rock_data = compute_rock_radii(results, params)


for rock in results:
    rock_points_rover_frame.append(
        project_pixel_to_rover(rock["left_centroid"], rock["depth"], "FrontLeft", cam_config)
    )

rock_points_rover_frame = np.array(rock_points_rover_frame)
print(rock_points_rover_frame)
plot_points_rover_frame(rock_points_rover_frame, color="red")
plot_rocks_rover_frame(rock_points_rover_frame, rock_data, color="blue")


## Arc path planning


In [36]:
planner = ArcPlanner()


arcs = planner.np_candidate_arcs

In [27]:
print(rock_points_rover_frame)
test_rock_points_rover_frame = np.array([[0.5, 0.5], [0.4, -0.4], [0.2, 0.3]])

NameError: name 'rock_points_rover_frame' is not defined

In [28]:
test_rock_data = [0.07, 0.06, 0.05]

In [41]:
# print(f"candidate arcs shape: {planner.np_candidate_arcs.shape}")
waypoint = np.array([4.25, 4.25])
# current_pose = np.eye(4)
test_rock_points_rover_frame = np.array([[]])
test_rock_data = []
control, best_arc, waypoint_local = planner.plan_arc(
    waypoint, current_pose, test_rock_points_rover_frame, test_rock_data
)
# (v,w), waypoint_local = planner.plan_arc(waypoint, current_pose, test_rock_points_rover_frame, test_rock_data)
# print(waypoint_local)
# fig = plot_rocks_rover_frame(
#     test_rock_points_rover_frame, test_rock_data, waypoint=waypoint_local, color="red"
# )
print(f"waypoint_local{waypoint_local}")
fig = go.Figure()
for arc in arcs:
    fig = plot_path_rover_frame(arc, fig=fig)
fig = plot_path_rover_frame(best_arc, color="green", waypoint=waypoint_local, fig=fig)

fig.show()
print(control)


self.vw[(0.2, -1.0), (0.2, -1.0), (0.2, -1.0), (0.2, -1.0), (0.2, -1.0), (0.2, -0.5), (0.2, -0.5), (0.2, -0.5), (0.2, -0.5), (0.2, -0.5), (0.2, 0.0), (0.2, 0.0), (0.2, 0.0), (0.2, 0.0), (0.2, 0.0), (0.2, 0.5), (0.2, 0.5), (0.2, 0.5), (0.2, 0.5), (0.2, 0.5), (0.2, 1.0), (0.2, 1.0), (0.2, 1.0), (0.2, 1.0), (0.2, 1.0)]
2
waypoint_local[-1.20142065 -8.27371501 -2.10916042  1.        ]


(0.2, -1.0)
